# Understanding Different Modes of Block Cipher 

Consider two messages, each of which owns n blocks: $M_1 = m_1||m_2||...||m_n$ and $M_2 = m'_1||m_2||...||m_n$, where $M_1$ and $M_2$ differ only in the first block. For instance, 

$M_1$ = "11335577 is my student ID. We will encrypt the message with different ways."  
$M_2$ = "22446688 is my student ID. We will encrypt the message with different ways."

Which of the following statements hold? Assume the same key is used for all encryption.

1) If $M_1$ and $M_2$ are encrypted using `Electronic Code Book (ECB)`, none of the cipher text blocks will repeat between each message.
2) If $M_1$ and $M_2$ are encrypted using `Cipher Block Chaining (CBC)`, but the same Initialization Vector (IV) is used for encrypting both messages, none of the cipher text blocks will repeat between each message.

In [2]:
M1 = b"11335577 is my student ID. We will encrypt the message in different ways."
M2 = b"22446688 is my student ID. We will encrypt the message in different ways."

In [1]:
!python -m pip install cryptography

Defaulting to user installation because normal site-packages is not writeable


## Available modes in OpenSSL

aes-[128|192|256]-cbc  128/192/256 bit AES in CBC mode  
aes[128|192|256]       Alias for aes-[128|192|256]-cbc  
aes-[128|192|256]-cfb  128/192/256 bit AES in 128 bit CFB mode  
aes-[128|192|256]-cfb1 128/192/256 bit AES in 1 bit CFB mode  
aes-[128|192|256]-cfb8 128/192/256 bit AES in 8 bit CFB mode  
aes-[128|192|256]-ctr  128/192/256 bit AES in CTR mode  
aes-[128|192|256]-ecb  128/192/256 bit AES in ECB mode  
aes-[128|192|256]-ofb  128/192/256 bit AES in OFB mode

In [3]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.padding import PKCS7

In [4]:
import random

SEED = 0
BLOCK_SIZE = 128

# seed
random.seed(0)

# generate random key and iv
KEY = bytes(random.randrange(256) for _ in range(BLOCK_SIZE // 8))
IV = bytes(random.randrange(256) for _ in range(BLOCK_SIZE // 8))
print(KEY.hex(), IV.hex())

c5d71484f8cf9bf4b76f47904730804b 9e3225a9f133b5dea168f4e2851f072f


In [5]:
# create padder that required by ECB and CBC modes, https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/
padder, unpadder = PKCS7(BLOCK_SIZE).padder(), PKCS7(BLOCK_SIZE).unpadder()
padded_data = padder.update(M1) + padder.finalize()
print(padded_data)
print(unpadder.update(padded_data) + unpadder.finalize())

padder, unpadder = PKCS7(BLOCK_SIZE).padder(), PKCS7(BLOCK_SIZE).unpadder()
padded_data = padder.update(M2) + padder.finalize()
print(padded_data)
print(unpadder.update(padded_data) + unpadder.finalize())

b'11335577 is my student ID. We will encrypt the message in different ways.\x07\x07\x07\x07\x07\x07\x07'
b'11335577 is my student ID. We will encrypt the message in different ways.'
b'22446688 is my student ID. We will encrypt the message in different ways.\x07\x07\x07\x07\x07\x07\x07'
b'22446688 is my student ID. We will encrypt the message in different ways.'


In [6]:
def encrypt_then_decrypt(plaintext, mode=modes.ECB()):
    cipher = Cipher(algorithms.AES(KEY), mode)
    encryptor, decryptor = cipher.encryptor(), cipher.decryptor()
    padder, unpadder = PKCS7(128).padder(), PKCS7(128).unpadder()

    # encrypt
    if len(plaintext) % 16 != 0:
        padded_data = padder.update(plaintext) + padder.finalize()
    else:
        padded_data = plaintext

    ciphered_data = encryptor.update(padded_data) + encryptor.finalize()

    # decrypt
    decipered_data = decryptor.update(ciphered_data) + decryptor.finalize()
    if len(plaintext) % 16 != 0:
        unpadded_data = unpadder.update(decipered_data) + unpadder.finalize()
        assert plaintext == unpadded_data

    return ciphered_data

In [7]:
import textwrap

color2num = dict(
    gray=30,
    red=31,
    green=32,
    yellow=33,
    blue=34,
    magenta=35,
    cyan=36,
    white=37,
    crimson=38,
)

def colorize(string, color, bold=True, highlight=False):
    """
    Colorize a string.

    This function was originally written by John Schulman.
    """
    attr = []
    num = color2num[color]
    if highlight:
        num += 10
    attr.append(str(num))
    if bold:
        attr.append("1")
    return "\x1b[%sm%s\x1b[0m" % (";".join(attr), string)

def visual_hex_diff(bstr_1, bstr_2, hex_names=("HEX 1", "HEX 2")):
    SEP = "   |   "
    print("  ", f"{hex_names[0]}".ljust(16 + 7), hex_names[1], sep=SEP)
    # block level
    hex_1, hex_2 = textwrap.wrap(bstr_1.hex(), 16), textwrap.wrap(bstr_2.hex(), 16)
    for i, (block_1, block_2) in enumerate(zip(hex_1, hex_2)):
        # byte level
        block_1, block_2 = textwrap.wrap(block_1, 2), textwrap.wrap(block_2, 2)
        block_2 = [colorize(v2, "red" if v1 != v2 else "green") for v1, v2 in zip(block_1, block_2)]
        print(str(i).rjust(2), " ".join(block_1).ljust(16 + 7), " ".join(block_2).ljust( 16 + 7), sep=SEP)

In [8]:
ciphered_M1 = encrypt_then_decrypt(M1, modes.ECB())
ciphered_M2 = encrypt_then_decrypt(M2, modes.ECB())
visual_hex_diff(ciphered_M1, ciphered_M2, ["deciphered M1", "deciphered M2"])

     |   deciphered M1             |   deciphered M2
 0   |   f9 ac 8d ae 3b fe f7 44   |   91 59 26 98 55 62 5c c3
 1   |   88 0c 3c 13 ee 28 ee b2   |   93 9c 01 87 87 7e c1 e0
 2   |   e4 1e 2f ad aa 39 39 21   |   e4 1e 2f ad aa 39 39 21
 3   |   55 21 cd f8 04 20 a7 d9   |   55 21 cd f8 04 20 a7 d9
 4   |   e5 ae 86 39 1e a0 18 5b   |   e5 ae 86 39 1e a0 18 5b
 5   |   be 9f da d5 16 05 d4 1b   |   be 9f da d5 16 05 d4 1b
 6   |   56 c8 43 97 48 49 ee f2   |   56 c8 43 97 48 49 ee f2
 7   |   81 87 6d 73 ac d9 9d 70   |   81 87 6d 73 ac d9 9d 70
 8   |   0d 7a 24 96 06 97 c9 6b   |   0d 7a 24 96 06 97 c9 6b
 9   |   fe 8d 61 39 26 93 c8 d7   |   fe 8d 61 39 26 93 c8 d7


In [9]:
ciphered_M1 = encrypt_then_decrypt(M1, modes.CBC(IV))
ciphered_M2 = encrypt_then_decrypt(M2, modes.CBC(IV))
visual_hex_diff(ciphered_M1, ciphered_M2, ["deciphered M1", "deciphered M2"])

     |   deciphered M1             |   deciphered M2
 0   |   51 0a 1f 95 1e 74 d2 bd   |   04 d7 5e 34 54 74 ef d6
 1   |   91 8c 60 7c 59 ec c7 08   |   c9 3c 7b 27 67 d0 3a 2c
 2   |   de f5 a0 a2 86 d7 8a 6f   |   fb 32 41 7b f8 a8 0a b3
 3   |   b5 68 ee fc 92 43 91 a9   |   bd 46 27 41 30 51 07 25
 4   |   81 73 cc 12 21 7f 0a 39   |   c5 aa 36 45 1c b2 f9 a4
 5   |   2a 73 82 19 ed 52 c6 01   |   22 ce 4a ac e2 da ed 57
 6   |   65 34 86 05 59 b1 c8 aa   |   a3 24 1f 83 26 8b 2f a3
 7   |   6a 82 f0 eb a8 bd 57 93   |   35 9a b8 d3 31 3e ed 73
 8   |   a4 a8 da 02 be ae ea 97   |   fc 69 79 5d c1 97 0d 57
 9   |   93 1c 5b 3b a9 8e 52 b1   |   97 4a 15 b3 cb da 9d a3
